# Using Applications

To run different protein design applications, we first have to import everything we need. In this case, we want to test the application ligandmpnn.

In [2]:
import sys
sys.path.append("..") # add the path to the protflow directory (e.g. /home/username/ProtSlurm)
from protflow.poses import Poses
from protflow.tools.ligandmpnn import LigandMPNN
from protflow.jobstarters import SbatchArrayJobstarter, LocalJobStarter

First, we define our jobstarters. We want to test if we can run LigandMPNN on GPU or cpu using the SLURM workload manager as well as run it locally. If SLURM is not installed on your machine, you can skip the parts mentioning it.

In [3]:
slurm_gpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=1)
slurm_cpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=False)

In [4]:
local_jobstarter = LocalJobStarter(max_cores=1)

Next, we have to load our poses. We set the local_jobstarter as default jobstarter.

In [5]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='ligandmpnn_local', storage_format='csv', jobstarter=local_jobstarter)

To run ligandmpnn, we have to create a runner. Make sure the path to the LigandMPNN script and python path are set in protflow/config.py! The lines there should look like:

#ligandmpnn.py

LIGANDMPNN_SCRIPT_PATH = "/home/user/LigandMPNN/run.py"

LIGANDMPNN_PYTHON_PATH = "/home/user/anaconda3/envs/ligandmpnn_env/bin/python3"

You can set it also when creating the runner, but it is recommended to set it in the config if you want to run it again.

If running this notebook on a cluster, be sure that is also opened from there (e.g. with VS code installed on the cluster)! Otherwise, the /home/ directories won't match since it will look for the files on your local machine and not on the cluster!

In [6]:
ligandmpnn = LigandMPNN()

To run ligandmpnn on our poses, we have to use the .run() function. All tools and metrics should have this function. It is mandatory to provide a unique prefix for each run. Each score generated will be saved to the poses dataframe in the format prefix_scorename. The output files can be found in a folder called prefix in the working_directory set for the input poses. The .run() function always returns poses.

In [ ]:
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_local', nseq=2, model_type='protein_mpnn')
display(my_poses.df)

Notice how the poses dataframe has changed! It now contains all the poses generated from LigandMPNN and the corresponding scores. 

Since we did not provide a jobstarter when we set up ligandmpnn, it ran on the local machine, because it defaulted to the jobstarter we set when creating our poses. We can run LigandMPNN using another jobstarter, either by providing LigandMPNN a default jobstarter or calling the .run function with the jobstarter option.

In [ ]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='ligandmpnn_cpu', storage_format='csv', jobstarter=local_jobstarter)
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_cpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_cpu_jobstarter)
display(my_poses.df)

In [ ]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='ligandmpnn_gpu', storage_format='csv', jobstarter=local_jobstarter)

gpu_ligandmpnn = LigandMPNN(jobstarter=slurm_gpu_jobstarter)
my_poses = gpu_ligandmpnn.run(poses=my_poses, prefix='ligmpnn_gpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_gpu_jobstarter)
display(my_poses.df)

You can provide options to your runner. These will be applied to all input poses. If you want to apply pose-specific options, you can use pose_options. The pose_options option reads in either a dataframe column (where each row contains the option for the corresponding pose) or a list of options for each pose. You can combine options and pose_options freely. If setting an option using both the generic options and pose_options, the pose_options will overwrite the generic one (but this should in general be avoided to prevent confusion). Some runners have explicit arguments for options that are mandatory or used very often (like model_type for LigandMPNN).

The options must be formatted in the same way as when running them directly. E.g. LigandMPNN uses options and flags with a -- prefix, whereas Rosetta uses options and flags indicated by -.

We are going to set general options to bias ProteinMPNN to prefer Ala residues and provide pose_options to preserve certain residues which should not be designed.

In [14]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='ligandmpnn_options', storage_format='csv', jobstarter=local_jobstarter)

pose_options = [f"--fixed_residues 'A{index+1}'" for index, pose in my_poses.df.iterrows()]
my_poses.df['mpnn_pose_options'] = pose_options
display(my_poses.df[['poses', 'mpnn_pose_options']])

my_poses = ligandmpnn.run(poses=my_poses, prefix='options_local', nseq=10, model_type='protein_mpnn', options='--bias_AA A:2', pose_options='mpnn_pose_options')
display(my_poses.df)

,poses,mpnn_pose_options
0,data/input_pdbs/structure1.pdb,--fixed_residues 'A1'
1,data/input_pdbs/structure3.pdb,--fixed_residues 'A2'
2,data/input_pdbs/structure2.pdb,--fixed_residues 'A3'


,input_poses,poses,poses_description,mpnn_pose_options,options_local_mpnn_origin,options_local_seed,options_local_description,options_local_sequence,options_local_T,options_local_id,options_local_seq_rec,options_local_overall_confidence,options_local_ligand_confidence,options_local_location
0,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0001,--fixed_residues 'A1',structure1,55092.0,structure1_0001,MAAAFAAALAAAAAAAAAAAAAAAAAAAPAAAAVAAAPAAAAAAAA...,0.1,1.0,0.2864,0.3198,0.3198,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
1,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0002,--fixed_residues 'A1',structure1,55092.0,structure1_0002,MAAAFAAAAAAAAADLAAAAAAAAAALAPAAAAVAANPAAAAAAAA...,0.1,2.0,0.2814,0.3090,0.3090,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
2,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0003,--fixed_residues 'A1',structure1,55092.0,structure1_0003,MAAAFAAAAAAAAAAVAAAAAAVAAALAAHAAAVAANPAAAATAAA...,0.1,3.0,0.3015,0.3421,0.3421,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
3,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0004,--fixed_residues 'A1',structure1,55092.0,structure1_0004,MAAAAAAAAAAAAAAAAANAAAAAAAAAPYAAAVAAAPAAAATAAA...,0.1,4.0,0.2915,0.3016,0.3016,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
4,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0005,--fixed_residues 'A1',structure1,55092.0,structure1_0005,MAAAAAAALAAAAAAAAAAAAAAAAAAAPYAAAVAANPAAAAAAAA...,0.1,5.0,0.2965,0.3232,0.3232,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
5,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0006,--fixed_residues 'A1',structure1,55092.0,structure1_0006,MAAAFAAALAAAAADAAAAAAAAAAALAAAAAAVAADAAAAAAAAA...,0.1,6.0,0.3065,0.3211,0.3211,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
6,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0007,--fixed_residues 'A1',structure1,55092.0,structure1_0007,MAAAAAAAAAAAAADLAAAAAAAAAALAAHIAAVRAAPAAAATAAA...,0.1,7.0,0.3166,0.3335,0.3335,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
7,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0008,--fixed_residues 'A1',structure1,55092.0,structure1_0008,MAAAAAAAAAALAAAIAAAAAAAAAAIAPYAAAVRANPAAAATAAA...,0.1,8.0,0.3015,0.3118,0.3118,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
8,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0009,--fixed_residues 'A1',structure1,55092.0,structure1_0009,MAAAFAAALAAAAADAAAAAAAVAAALAPYAAAVAAAPAALATFRA...,0.1,9.0,0.3467,0.3332,0.3332,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
9,data/input_pdbs/structure1.pdb,/home/tripp/ProtFlow/examples/ligandmpnn_optio...,structure1_0010,--fixed_residues 'A1',structure1,55092.0,structure1_0010,MAAAAAAALAALAAAAAAAAAAAAAALAPAAAAVAANPAAAAAAAA...,0.1,10.0,0.3065,0.3208,0.3208,/home/tripp/ProtFlow/examples/ligandmpnn_optio...
